In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from pylab import rcParams

In [19]:
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge

In [20]:
#Change the path according to the location of files
test_volume = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/testing_phase1/test1_20min_avg_volume.csv", names=["tollgate_id","time_window","direction","volume"])
test_travel_time = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/testing_phase1/test1_20min_avg_travel_time.csv", names = ["intersection_id","tollgate_id","time_window","avg_travel_time"])
training_volume = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/training/training_20min_avg_volume.csv", names=["tollgate_id","time_window","direction","volume"])
training_travel_time = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/training/training_20min_avg_travel_time.csv", names = ["intersection_id","tollgate_id","time_window","avg_travel_time"])

In [21]:
#Generate a general vector
def generate_vector(data_frame, data_type):
    P_expanded = data_frame['time_window'].apply(lambda x: pd.Series(x.split(',')))
    P_expanded[1] = P_expanded[1].apply(str).apply(lambda x : pd.Series(x.rstrip(')')))
    P_expanded[0] = P_expanded[0].apply(str).apply(lambda x : pd.Series(x.lstrip('[')))
    data_frame = data_frame.join(P_expanded)
    del data_frame["time_window"]
    if (data_type == "volume"):
        data_frame = data_frame[["tollgate_id",0,1,"direction", "volume"]] 
    else:
        data_frame = data_frame[["intersection_id","tollgate_id",0,1,"avg_travel_time"]]
    return data_frame

In [22]:
training_volume = generate_vector(training_volume, "volume")
test_travel_time = generate_vector(test_travel_time, "travel_time")
test_volume = generate_vector(test_volume, "volume")
training_travel_time = generate_vector(training_travel_time, "travel_time")

In [23]:
#Change the path according to the location of files
training_weather_data = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/testing_phase1/weather (table 7)_test1.csv", names=["date","hour","pressure","sea_pressure","wind_direction","wind_speed","temperature","rel_humidity","precipitation"])
testing_weather_data = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/training/weather (table 7)_training.csv", names=["date","hour","pressure","sea_pressure","wind_direction","wind_speed","temperature","rel_humidity","precipitation"])
routes = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/training/routes (table 4).csv")
links = pd.read_csv("C:/Users/Shruti1481/PycharmProjects/HighwayTrafficPrediction/DATA/dataSets/dataSets/training/links (table 3).csv")
links = links.fillna(0)

In [24]:
#process links file
links[["in_top","out_top"]] = links[["in_top","out_top"]].values.astype(str)
in_top = links['in_top'].apply(lambda x: pd.Series(x.split(',')))
out_top = links['out_top'].apply(lambda x: pd.Series(x.split(',')))
links[["in_top_0","in_top_1"]] = in_top
links[["out_top_0","out_top_1"]] = out_top
del links['in_top']
del links['out_top']
links = links [['link_id', 'length', 'width', 'lanes', 'in_top_0', 'in_top_1', 'out_top_0', 'out_top_1',
       'lane_width']]


In [25]:
def get_volume_vector(volume_data, weather_data):
    weather_data["hour"] = weather_data["hour"].apply(str)
    weather_data["hour"] = weather_data["hour"] + ":00"
    weather_data[0] = weather_data['date'] + ' ' + weather_data['hour']
    del weather_data["date"]
    del weather_data["hour"]
    weather_data[0] = pd.to_datetime(weather_data[0])
    volume_data[0] = pd.to_datetime(volume_data[0]) 
    volume_vector = volume_data.join(weather_data.set_index(0).reindex(volume_data.set_index(0).index, method='nearest').reset_index(), how='right',  lsuffix='_start', rsuffix='_weather')
    del volume_vector["0_weather"]
    new_columns = volume_vector.columns.values
    new_columns[1] = 0
    volume_vector.columns = new_columns
    volume_vector = volume_vector[['tollgate_id', 0, 1, 'direction', 'pressure',
       'sea_pressure', 'wind_direction', 'wind_speed', 'temperature',
       'rel_humidity', 'precipitation', 'volume']]
    return volume_vector    


In [26]:
def process_volume_vector(volume_vector):
    #to float direction
    volume_vector["direction"] = volume_vector["direction"].values.astype(float) 

    #standardize other columns
    x = volume_vector[["pressure", "sea_pressure", "wind_direction","temperature", "rel_humidity"]].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x.T)
    volume_vector_normalized = pd.DataFrame(x_scaled.T)

    volume_vector["pressure"] = volume_vector_normalized[0]
    volume_vector["sea_pressure"] = volume_vector_normalized[1]
    volume_vector["wind_direction"] = volume_vector_normalized[2]
    volume_vector["rel_humidity"] = volume_vector_normalized[3]
    
    del volume_vector[1]
    volume_vector = volume_vector.assign(time_window=20)
    
    #preprocess time
    x = volume_vector[0].values.astype(float)
    x = x.reshape((len(x), 1))
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 2))
    y = scaler.fit_transform(x)
    volume_vector[0] = pd.DataFrame(y)
    volume_vector = volume_vector[['tollgate_id', 0, 'time_window', 'direction', 'pressure', 'sea_pressure',
       'wind_direction', 'wind_speed', 'temperature', 'rel_humidity',
       'precipitation', 'volume']]
    return volume_vector
    

In [27]:
#Get the unprocessed vectors
train_volume_vector = get_volume_vector(training_volume,training_weather_data)
test_volume_vector = get_volume_vector(test_volume,testing_weather_data)

In [28]:
#Standardize the vector data
train_volume_vector = process_volume_vector(train_volume_vector)
test_volume_vector = process_volume_vector(test_volume_vector)

In [29]:
def add_links(vector, vector_expanded, name, number):
    vector[name] = vector_expanded[number]
    vector[name] = pd.to_numeric(vector[name])
    vector = pd.merge(vector, links, how='inner', left_on=[name], right_on=['link_id'])
    return vector  

In [30]:
def get_travel_time_vector(travel_vector, weather_data):
    travel_vector[0] = pd.to_datetime(travel_vector[0]) 
    travel_vector['avg_travel_time'] = pd.to_numeric(travel_vector['avg_travel_time'])
    travel_vector_joined = travel_vector.join(weather_data.set_index(0).reindex(travel_vector.set_index(0).index, method='nearest').reset_index(), how='right',  lsuffix='_start', rsuffix='_weather')
    travel_vector_joined = pd.merge(travel_vector_joined, routes, how='inner', on=['intersection_id', 'tollgate_id'])
    travel_vector_joined_expanded = travel_vector_joined['link_seq'].apply(lambda x: pd.Series(x.split(',')))
   
    links['link_id'] =  pd.to_numeric(links['link_id']) 

    travel_vector_joined = add_links(travel_vector_joined, travel_vector_joined_expanded, 'first_link_id', 0 )
    travel_vector_joined = add_links(travel_vector_joined, travel_vector_joined_expanded, 'second_link_id', 1 )
    travel_vector_joined = add_links(travel_vector_joined, travel_vector_joined_expanded, 'third_link_id', 2 )
    travel_vector_joined = add_links(travel_vector_joined, travel_vector_joined_expanded, 'fourth_link_id', 3 )
    travel_vector_joined = add_links(travel_vector_joined, travel_vector_joined_expanded, 'fifth_link_id', 4 )
    
    travel_vector_joined = travel_vector_joined.assign(time_window = 20)
    del travel_vector_joined['0_weather']
    del travel_vector_joined['link_seq']
    del travel_vector_joined['link_id_x']
    del travel_vector_joined['link_id_y']
    del travel_vector_joined['link_id']
    del travel_vector_joined[1]
    
    travel_vector_joined = travel_vector_joined[['intersection_id', 'tollgate_id', '0_start', 'time_window',
       'pressure', 'sea_pressure', 'wind_direction', 'wind_speed',
       'temperature', 'rel_humidity', 'precipitation', 'first_link_id',
       'length_x', 'width_x', 'lanes_x', 'in_top_0_x', 'in_top_1_x',
       'out_top_0_x', 'out_top_1_x', 'lane_width_x', 'second_link_id',
       'length_y', 'width_y', 'lanes_y', 'in_top_0_y', 'in_top_1_y',
       'out_top_0_y', 'out_top_1_y', 'lane_width_y', 'third_link_id',
       'length_x', 'width_x', 'lanes_x', 'in_top_0_x', 'in_top_1_x',
       'out_top_0_x', 'out_top_1_x', 'lane_width_x', 'fourth_link_id',
       'length_y', 'width_y', 'lanes_y', 'in_top_0_y', 'in_top_1_y',
       'out_top_0_y', 'out_top_1_y', 'lane_width_y', 'fifth_link_id',
       'length', 'width', 'lanes', 'in_top_0', 'in_top_1', 'out_top_0',
       'out_top_1', 'lane_width', 'avg_travel_time']]
    
    
    return travel_vector_joined

In [31]:
train_travel_time = get_travel_time_vector(training_travel_time,training_weather_data)
test_travel_time = get_travel_time_vector(test_travel_time,testing_weather_data)

In [32]:
def process_travel_data(travel_vector):
    le = LabelEncoder()
    df = pd.DataFrame(travel_vector['intersection_id'], columns=['intersection_id'])
    encoded_series = df[df.columns[:]].apply(le.fit_transform)
    travel_vector['intersection_id'] = encoded_series
    
    #standardize other columns
    x = travel_vector[["pressure", "sea_pressure", "wind_direction","temperature", "rel_humidity"]].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x.T)
    travel_vector_normalized = pd.DataFrame(x_scaled.T)

    travel_vector["pressure"] = travel_vector_normalized[0]
    travel_vector["sea_pressure"] = travel_vector_normalized[1]
    travel_vector["wind_direction"] = travel_vector_normalized[2]
    travel_vector["rel_humidity"] = travel_vector_normalized[3]
    
   
    
    #preprocess time
    x = travel_vector['0_start'].values.astype(float)
    x = x.reshape((len(x), 1))
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 2))
    y = scaler.fit_transform(x)
    travel_vector['0_start'] = pd.DataFrame(y)
    
    travel_vector = travel_vector.fillna(0)
   
    return travel_vector

    

In [33]:
training_travel_vector_processed = process_travel_data(train_travel_time)
testing_travel_vector_processed = process_travel_data(test_travel_time)

In [34]:
#Finding the MAPE for travel time
def mean_absolute_percentage_error_travel(travel_vector, actual_travel_time, predicted_travel_time):
    travel_time = {}
    for i in range(len(travel_vector)):
        results = abs((actual_travel_time[i] - predicted_travel_time[i]) / actual_travel_time[i])  
        key = str(int(travel_vector['intersection_id'][i])) + "-" + str(int(travel_vector['tollgate_id'][i]))
        if key not in travel_time:
            travel_time[key] = {}
        time_window = str(travel_vector['0_start'][i]) 
        
        if time_window not in travel_time[key]:
            travel_time[key][time_window] = results
        else:
            travel_time[key][time_window] += results

    tollgate_number = len(travel_time)
    mape = 0.0
    for detail in travel_time:
        tollgate_number_content = len(travel_time[detail])
        total = 0.0
        for time in travel_time[detail]:
            total += travel_time[detail][time]
        mape += (total) / tollgate_number_content
    mape = mape / tollgate_number
    

    return mape

In [35]:
def mean_absolute_percentage_error_volume(volume_vector, actual_volume, predicted_volume):
    volume = {}
    for i in range(len(volume_vector)):
        result = abs((actual_volume[i] - predicted_volume[i]) / actual_volume[i]) # resultult of current instance
        key = str(int(volume_vector['tollgate_id'][i])) + "-" + str(int(volume_vector['direction'][i]))
        if key not in volume:
            volume[key] = {}
        time_window = str(volume_vector['tollgate_id'][i]) + "-" + str(volume_vector[0][i])
        if time_window not in volume[key]:
            # Current Timegate doesn't exist
            volume[key][time_window] = result
        else:
            # Current Timegate exists
            volume[key][time_window] += result

    
    mape = 0.0
    tollgate_number = len(volume)
    for detail in volume:
        tollgate_number_content = len(volume[detail])
        tempSum = 0.0
        for time in volume[detail]:
            tempSum += volume[detail][time]
        mape += (tempSum)/tollgate_number_content
    mape = mape / tollgate_number
    return mape


In [36]:
train_volume_x = train_volume_vector.loc[:,train_volume_vector.columns != 'volume']
train_volume_y = train_volume_vector['volume'] 
test_volume_x = test_volume_vector.loc[:,test_volume_vector.columns != 'volume']
test_volume_y = test_volume_vector['volume'] 
train_travel_x = training_travel_vector_processed.loc[:,training_travel_vector_processed.columns != 'avg_travel_time']
train_travel_y = training_travel_vector_processed['avg_travel_time'] 
test_travel_x = testing_travel_vector_processed.loc[:,testing_travel_vector_processed.columns != 'avg_travel_time']
test_travel_y = testing_travel_vector_processed['avg_travel_time']

In [37]:
###Decision Tree Regressor###
def decisionTreeRegressor_volume():
    volume_regression = DecisionTreeRegressor()
  
    #train the model to predict volume and calculate errors
    volume_regression.fit(train_volume_x, train_volume_y)
    predicted_volume = volume_regression.predict(test_volume_x)
    volume_mean_square_error = mean_squared_error(predicted_volume,test_volume_y)
    volume_absolute_error = mean_absolute_error(predicted_volume,test_volume_y)
    error = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,predicted_volume)
    return volume_mean_square_error,volume_absolute_error, error


In [38]:
def decisionTreeRegressor_travel_time():
    travel_time_regression = DecisionTreeRegressor()
    
    #train the model to predict travel time and calculate errors
    travel_time_regression.fit(train_travel_x, train_travel_y)
    predicted_travel_time = travel_time_regression.predict(test_travel_x)
    time_mse = mean_squared_error(predicted_travel_time,test_travel_y)
    time_mae = mean_absolute_error(predicted_travel_time,test_travel_y)
    time_mape = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,predicted_travel_time)
    return time_mse,time_mae,time_mape

In [39]:
volume_mse_DT,volume_mae_DT,volume_mape_DT = decisionTreeRegressor_volume()
time_mse_DT,time_mae_DT,time_mape_DT = decisionTreeRegressor_travel_time()

In [40]:
print('volume:',volume_mse_DT,volume_mae_DT,volume_mape_DT)
print('travel:',time_mse_DT,time_mae_DT,time_mape_DT)

volume: 3361.97142857 46.8142857143 0.914565112321
travel: 5641.92102634 42.1175446429 0.331464636469


In [41]:
###SVR Volume####
def SVR_volume():
    svr_clf = SVR(C=1.0, epsilon=0.2)
    svr_clf.fit(train_volume_x,train_volume_y)
    svr_predicted_volume = svr_clf.predict(test_volume_x)
    volume_mean_square_error_svr = mean_squared_error(svr_predicted_volume,test_volume_y)
    volume_absolute_error_svr = mean_absolute_error(svr_predicted_volume,test_volume_y)
    error_svr = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,svr_predicted_volume)
    return volume_mean_square_error_svr,volume_absolute_error_svr, error_svr

In [42]:
###SVR Travel time####
def SVR_travel_time():
    svr_clf = SVR(C=1.0, epsilon=0.2)
    svr_clf.fit(train_travel_x, train_travel_y)
    svr_predicted_travel_time = svr_clf.predict(test_travel_x)
    travel_time_mse_svr = mean_squared_error(svr_predicted_travel_time,test_travel_y)
    travel_time_mae_svr = mean_absolute_error(svr_predicted_travel_time,test_travel_y)
    travel_time_mape_svr = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,svr_predicted_travel_time)
    return travel_time_mse_svr,travel_time_mae_svr, travel_time_mape_svr

In [43]:
volume_mse_SVR,volume_mae_SVR,volume_mape_SVR = SVR_volume()
time_mse_SVR,time_mae_SVR,time_mape_SVR = SVR_travel_time()

In [44]:
print('volume:', volume_mse_SVR,volume_mae_SVR,volume_mape_SVR)
print('travel:', time_mse_SVR,time_mae_SVR,time_mape_SVR)

volume: 1661.8157026 32.9731943046 0.5586230289
travel: 2099.33590758 26.5254086869 0.201875255101


In [45]:
## Bayesian Ridge volume prediction ##
def BayesianRidge_volume():
    bay_clf = linear_model.BayesianRidge()
    bay_clf.fit(train_volume_x,train_volume_y)
    bay_predicted_volume = bay_clf.predict(test_volume_x)
    bay_clf = linear_model.BayesianRidge()
    bay_clf.fit(train_volume_x,train_volume_y)
    bay_predicted_volume = bay_clf.predict(test_volume_x)
    volume_mse_bay = mean_squared_error(bay_predicted_volume,test_volume_y)
    volume_mae_bay = mean_absolute_error(bay_predicted_volume,test_volume_y)
    volume_mape_bay = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,bay_predicted_volume)
    return volume_mse_bay,volume_mae_bay, volume_mape_bay 

In [46]:
###Bayesian Ridge Travel time####
def BayesianRidge_travel_time():
    bay_clf = linear_model.BayesianRidge()
    bay_clf.fit(train_travel_x, train_travel_y)
    bay_predicted_travel_time = bay_clf.predict(test_travel_x)
    travel_time_mse_bay = mean_squared_error(bay_predicted_travel_time,test_travel_y)
    travel_time_mae_bay = mean_absolute_error(bay_predicted_travel_time,test_travel_y)
    travel_time_mape_bay = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,bay_predicted_travel_time)
    return travel_time_mse_bay,travel_time_mae_bay, travel_time_mape_bay 

In [47]:

volume_mse_BR,volume_mae_BR,volume_mape_BR = BayesianRidge_volume()
time_mse_BR,time_mae_BR,time_mape_BR = BayesianRidge_travel_time()

In [48]:
print('volume:', volume_mse_BR,volume_mae_BR,volume_mape_BR)
print('travel:', time_mse_BR,time_mae_BR,time_mape_BR)

volume: 1465.3170762 31.2057339491 0.615976677619
travel: 1781.26996322 25.3077006077 0.210225140542


In [49]:
## K Nearest Neighbours Regression volume prediction ##
def KNN_volume():
    knn_clf = KNeighborsRegressor(n_neighbors =2500)
    knn_clf.fit(train_volume_x,train_volume_y)
    knn_predicted_volume = knn_clf.predict(test_volume_x)
    volume_mse_knn = mean_squared_error(knn_predicted_volume,test_volume_y)
    volume_mae_knn = mean_absolute_error(knn_predicted_volume,test_volume_y)
    volume_mape_knn = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,knn_predicted_volume)
    return volume_mse_knn,volume_mae_knn, volume_mape_knn  

In [50]:
###K Nearest Neighbours Regression Travel time####
def KNN_travel_time():
    knn_clf = KNeighborsRegressor(n_neighbors =2500)
    knn_clf.fit(train_travel_x, train_travel_y)
    knn_predicted_travel_time = knn_clf.predict(test_travel_x)
    travel_time_mse_knn = mean_squared_error(knn_predicted_travel_time,test_travel_y)
    travel_time_mae_knn = mean_absolute_error(knn_predicted_travel_time,test_travel_y)
    travel_time_mape_knn = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,knn_predicted_travel_time)
    return travel_time_mse_knn,travel_time_mae_knn, travel_time_mape_knn 
    
    

In [51]:
volume_mse_KNN,volume_mae_KNN,volume_mape_KNN = KNN_volume()
time_mse_KNN,time_mae_KNN,time_mape_KNN = KNN_travel_time()

In [52]:
print('volume:', volume_mse_KNN,volume_mae_KNN,volume_mape_KNN)
print('travel:', time_mse_KNN,time_mae_KNN,time_mape_KNN)

volume: 1418.96856347 31.4203228571 0.632595341123
travel: 2204.57691387 28.4908805893 0.229760059494


In [53]:
## Random Forest Regression volume prediction ##
def RF_volume():
    rf_clf = RandomForestRegressor(max_depth=3, random_state=1)
    rf_clf.fit(train_volume_x,train_volume_y)
    rf_predicted_volume = rf_clf.predict(test_volume_x)
    volume_mse_rf = mean_squared_error(rf_predicted_volume,test_volume_y)
    volume_mae_rf = mean_absolute_error(rf_predicted_volume,test_volume_y)
    volume_mape_rf = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,rf_predicted_volume)
    return volume_mse_rf,volume_mae_rf, volume_mape_rf 


In [54]:
###RandomForest Travel time####
def RF_travel_time():
    rf_clf = RandomForestRegressor(max_depth=3, random_state=1)
    rf_clf.fit(train_travel_x, train_travel_y)
    rf_predicted_travel_time = rf_clf.predict(test_travel_x)
    travel_time_mse_rf = mean_squared_error(rf_predicted_travel_time,test_travel_y)
    travel_time_mae_rf = mean_absolute_error(rf_predicted_travel_time,test_travel_y)
    travel_time_mape_rf = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,rf_predicted_travel_time)
    return travel_time_mse_rf,travel_time_mae_rf, travel_time_mape_rf 
    

In [55]:
volume_mse_RF,volume_mae_RF,volume_mape_RF = RF_volume()
time_mse_RF,time_mae_RF,time_mape_RF = RF_travel_time()

In [56]:
print('volume:', volume_mse_RF,volume_mae_RF,volume_mape_RF)
print('travel:', time_mse_RF,time_mae_RF,time_mape_RF)

volume: 1413.57807453 31.1882618011 0.597842390574
travel: 1845.66189727 25.8834434225 0.211492883615


In [57]:
## Bagging Regressor volume prediction ##
def Bagging_volume():
    bg_clf = BaggingRegressor(base_estimator=SVR(C=1.0, epsilon=0.2),
                           n_estimators=50,
                           bootstrap=True,
                           oob_score=True)
    bg_clf.fit(train_volume_x,train_volume_y)
    bg_predicted_volume = bg_clf.predict(test_volume_x)
    volume_mse_bg = mean_squared_error(bg_predicted_volume,test_volume_y)
    volume_mae_bg = mean_absolute_error(bg_predicted_volume,test_volume_y)
    volume_mape_bg = mean_absolute_percentage_error_volume(test_volume_x,test_volume_y ,bg_predicted_volume)
   
    return volume_mse_bg,volume_mae_bg, volume_mape_bg 

In [58]:
###Bagging Regressor Travel time####
def Bagging_travel_time():
    bg_clf = BaggingRegressor(base_estimator=SVR(C=1.0, epsilon=0.2),
                           n_estimators=50,
                           bootstrap=True,
                           oob_score=True)
    bg_clf.fit(train_travel_x, train_travel_y)
    bg_predicted_travel_time = bg_clf.predict(test_travel_x)
    travel_time_mse_bg = mean_squared_error(bg_predicted_travel_time,test_travel_y)
    travel_time_mae_bg = mean_absolute_error(bg_predicted_travel_time,test_travel_y)
    travel_time_mape_bg = mean_absolute_percentage_error_travel(test_travel_x,test_travel_y ,bg_predicted_travel_time)
    return travel_time_mse_bg,travel_time_mae_bg, travel_time_mape_bg 

In [ ]:
volume_mse_bag,volume_mae_bag,volume_mape_bag = Bagging_volume()
time_mse_bag,time_mae_bag,time_mape_bag = Bagging_travel_time()

In [ ]:
print('volume:', volume_mse_bag,volume_mae_bag,volume_mape_bag)
print('travel:', time_mse_bag,time_mae_bag,time_mape_bag)

In [ ]:
results_volume = []
results_volume.append(volume_mape_DT)
results_volume.append(volume_mape_RF)
results_volume.append(volume_mape_BR)
results_volume.append(volume_mape_SVR)
results_volume.append(volume_mape_KNN)
results_volume.append(volume_mape_bag)
results_travel = []
results_travel.append(time_mape_DT)
results_travel.append(time_mape_RF)
results_travel.append(time_mape_BR)
results_travel.append(time_mape_SVR)
results_travel.append(time_mape_KNN)
results_travel.append(time_mape_bag)

In [ ]:
##plot for volume for 20 mins

n_groups = 1

 
# create plot

fig, ax = plt.subplots()
rcParams['figure.figsize'] = 12,10
index = np.arange(n_groups)
bar_width = 0.2
opacity = 0.8
 
rects1 = plt.bar(index, results_volume[1], bar_width,
                 alpha=opacity,
                 color='b',
                 label='Random Forest')
 
rects2 = plt.bar(index + bar_width, results_volume[2], bar_width,
                 alpha=opacity,
                 color='g',
                 label='Bayesian Regression ')

rects3 = plt.bar(index + 2 *bar_width, results_volume[3], bar_width,
                 alpha=opacity,
                 color='r',
                 label='SVR')

rects5 = plt.bar(index + 3 *bar_width, results_volume[5], bar_width,
                 alpha=opacity,
                 color='pink',
                 label='Bagging Regressor')

rects3 = plt.bar(index + 4 *bar_width, results_volume[0], bar_width,
                 alpha=opacity,
                 color='y',
                 label='Decision Tree Regression')

rects4 = plt.bar(index + 5 *bar_width, results_volume[4], bar_width,
                 alpha=opacity,
                 color='black',
                 label='KNN')



 
 
plt.xlabel('Algorithms')
plt.ylabel('MAPE')
plt.title('Algorithms Comparison Volume')
plt.xticks(index + bar_width *2, ())
plt.legend()
 
plt.tight_layout()
plt.show()

In [ ]:
##Plot for Travel time
n_groups = 1

 
# create plot

fig, ax = plt.subplots()
rcParams['figure.figsize'] = 12,10
index = np.arange(n_groups)
bar_width = 0.2
opacity = 0.8
 
rects1 = plt.bar(index, results_travel[1], bar_width,
                 alpha=opacity,
                 color='b',
                 label='Random Forest')
 
rects2 = plt.bar(index + bar_width, results_travel[2], bar_width,
                 alpha=opacity,
                 color='g',
                 label='Bayesian Regression ')

rects3 = plt.bar(index + 2 *bar_width, results_travel[3], bar_width,
                 alpha=opacity,
                 color='r',
                 label='SVR')

rects3 = plt.bar(index + 3 *bar_width, results_travel[0], bar_width,
                 alpha=opacity,
                 color='y',
                 label='Decision Tree Regression')

rects4 = plt.bar(index + 4 *bar_width, results_travel[4], bar_width,
                 alpha=opacity,
                 color='black',
                 label='KNN')

rects5 = plt.bar(index + 5 *bar_width, results_travel[5], bar_width,
                 alpha=opacity,
                 color='pink',
                 label='Bagging Regressor')

 
 
plt.xlabel('Algorithms')
plt.ylabel('MAPE')
plt.title('Algorithms Comparison Travel time')
plt.xticks(index + bar_width *2, ())
plt.legend()
 
plt.tight_layout()
plt.show()